# Introduction to Building LLM Agents with Tools and Tracing

<!--- @wandbcode{fc-london-workshop-2025} -->

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Running the agent on a multi-step task.

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!pip install weave openai
```


In [1]:
# Global Configuration & Setup
import os
import inspect
import json
import weave # Must import weave before litellm for auto-patching
import openai
from enum import Enum
from pydantic import BaseModel, Field
from rich.panel import Panel
from rich.markdown import Markdown
from rich.console import Console as RichConsole
from exa_py import Exa
from typing import Any, Callable, Dict, List, get_type_hints

Define a model to use, as we are going to use tool calling you need a capable model like `Kimi-K2` or `GLM`

In [ ]:
os.environ["WANDB_API_KEY"] = ""
os.environ["EXA_API_KEY"] = ""


In [3]:
class Console(RichConsole):
    def md(self, text): 
        return self.print(Markdown(text))

console = Console()

MODEL_SMALL = "Qwen/Qwen3-235B-A22B-Instruct-2507"
MODEL_MEDIUM = "zai-org/GLM-4.5"
MODEL_LARGE = "moonshotai/Kimi-K2-Instruct"

WANDB_ENTITY = "wandb-applied-ai-team"
WANDB_PROJECT = "london-workshop-2025"

oai_client = openai.OpenAI(
    base_url='https://api.inference.wandb.ai/v1',
    api_key=os.getenv("WANDB_API_KEY"),
    project=f"{WANDB_ENTITY}/{WANDB_PROJECT}")
exa_client = Exa(api_key=os.getenv("EXA_API_KEY"))

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [4]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

weave: Logged in as Weights & Biases user: morgan.
weave: View Weave data at https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/weave


## 1. Basic LLM Call with OpenAI SDK

Let's start with a simple call to the LLM using/

![](images/01_trace.png)

In [5]:
# Define a simple message list (conversation history)
messages = [{"role": "user", "content": "Hello, LLM! How does an AI agent work?"}]

# Make the call
response = resp = oai_client.chat.completions.create(
    model = MODEL_SMALL,
    messages=messages,
)
# Print the response content
assistant_response = response.choices[0].message.content
console.md(f"LLM Response:\\n{assistant_response}")

# Click on the 🍩 linke below to see the trace in Weave 👇

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f724-8f3a-723e-aef2-d9d23872b6e3


LLM Response:\nHello! Great question! 😊                                                                           

An AI agent is a software system that perceives its environment, makes decisions, and takes actions to achieve     
specific goals. Think of it like a digital "entity" that can act autonomously—like a robot, a chatbot, or even a   
self-driving car—but often running purely in software.                                                             

Here’s a breakdown of how an AI agent works:                                                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   1. Perception                                                   

The agent gathers information from its environment using sensors or inputs. This could be:                         

 • Text input (like your message to me)                                                                            
 • Camera images (for a robot)                                                                                     
 • Sensor data (temperature, GPS, etc.)                                                                            
 • Data from databases or APIs                                                                                     

👉 Example: A customer service chatbot reads your message.                                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                          2. Processing & Decision-Making                                          

The agent uses AI models (like machine learning or rule-based systems) to interpret the input and decide what to   
do.                                                                                                                

This often involves:                                                                                               

 • Understanding context (e.g., natural language processing)                                                       
 • Reasoning (e.g., planning steps to solve a problem)                                                             
 • Learning from past experiences (if it's a learning agent)                                                       

👉 Example: The chatbot understands you're asking about AI agents and decides to explain step-by-step.             

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     3. Action                                                     

The agent performs an action using actuators or output mechanisms:                                                 

 • Sending a message                                                                                               
 • Moving a robot arm                                                                                              
 • Adjusting thermostat settings                                                                                   
 • Recommending a product                                                                                          

👉 Example: The chatbot replies with this explanation!                                                             

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                             4. Goal-Oriented Behavior                                             

A good AI agent is goal-driven. It doesn’t just react—it tries to achieve objectives, like:                        

 • Minimizing user wait time                                                                                       
 • Winning a game (like AlphaGo)          

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f724-bb95-78a9-9716-75c78a988052
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f724-f5fc-75e2-8ee7-4325f2c92191
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f724-f605-762f-a05b-50933ebb513f
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f725-7f1f-7a2a-9ba0-573ef7b1b4a4
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f725-8432-7118-87ad-8e525ba7d957
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f725-b655-70f8-bcc3-a79fa00eafb4


Because we imported `weave` and called `weave.init()`, the OpenAI SDK call above was automatically traced. You can open your Weave dashboard and see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [6]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.

@weave.op
def call_model(model_name: str, messages: List[Dict[str, Any]], **kwargs) -> str:
    "Call a model with the given messages and kwargs."
    response = oai_client.chat.completions.create(
        model=model_name,
        messages=messages,
        **kwargs
    )

    return response.choices[0].message

response = call_model(model_name=MODEL_SMALL, messages=messages)
console.md(response.content)

Hello! Great question! 😊                                                                                          

An AI agent is a software system that perceives its environment, makes decisions, and takes actions to achieve     
specific goals. Think of it like a digital "entity" that can observe, think, and act — much like a person or robot,
but in a virtual or computational space.                                                                           

Here’s a breakdown of how an AI agent works:                                                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   1. Perception                                                   

The agent gathers information from its environment using sensors or data inputs.                                   

 • Examples:                                                                                                       
    • A chatbot reads your text message.                                                                           
    • A self-driving car uses cameras and lidar to see the road.                                                   
    • A trading bot monitors stock prices.                                                                         

This input is processed to understand the current state of the world.                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     2. Decision-Making (Reasoning & Planning)                                     

Based on what it perceives, the agent decides what to do next. This is where AI techniques come in:                

 • Rule-based logic: If X happens, do Y.                                                                           
 • Machine learning models: Predict outcomes or choose actions based on patterns.                                  
 • Search & optimization: Explore possible actions to find the best one.                                           
 • Large Language Models (LLMs): Generate responses or plans using learned knowledge (like me!).                   

The agent may:                                                                                                     

 • Choose a single action.                                                                                         
 • Formulate a sequence of steps (a plan).                                                                         
 • Predict the consequences of actions.                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     3. Action                                                     

The agent performs an action that affects its environment.                                                         

 • Examples:                                                                                                       
    • A chatbot sends a reply.                                                                                     
    • A robot arm picks up an object.                                                                              
    • A recommendation system suggests a movie.                                                                    

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                        4. Learning (Optional but Powerful)                                        

Many advanced agents learn from experience to improve over time.                                                   

 • They use techniques like:            

![](images/02_nested_trace.png)


## 2. Introducing Tool Calling

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.

Check the Mistral docs for function calling: https://platform.openai.com/docs/guides/function-calling

![](images/function-calling-diagram-steps.png)

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [7]:
@weave.op 
def add_numbers(a: int, b: int) -> int:
    """Use this tool to add numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [8]:
add_numbers(1, 2)

3

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f725-a8c9-7ca1-972f-5e7ae8862870
weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f726-220b-790f-950a-3287889ac5e5


In [9]:
# this doesn't work...
call_model(model_name=MODEL_SMALL, messages=messages, tools=[add_numbers])

TypeError: Object of type function is not JSON serializable

> We need to manually create the JSON schema describing this tool in a format that models *Mistral* understand.

In [10]:
# Manually define the tool schema
tool_add_numbers_schema = {
    "type": "function",
    "function": {
        "name": "add_numbers",
        "description": "Adds two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "The first number."
                },
                "b": {
                    "type": "integer",
                    "description": "The second number."
                }
            },
            "required": ["a", "b"]
        }
    }
}

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [11]:
messages = [
    {"role": "system", "content": "You are a helpful assistant use tools to answer questions."},
    {"role": "user", "content": "My lucky numbers are 77 and 11. What is their sum?"}]
response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[tool_add_numbers_schema])
console.print(response)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-6f223df227274e9fb786d71a22744087',
            function=Function(arguments='{"a": 77, "b": 11}', name='add_numbers'),
            type='function'
        )
    ],
    reasoning_content=None
)

## Manual Tool Call
The LLM's response might contain a request to call our tool (`response.choices[0].message.tool_calls`) or it might respond directly (`response.choices[0].message.content`). If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [12]:
if response.tool_calls:
    console.print("LLM requested a tool call:")
    for tool_call in response.tool_calls:
        function_name = tool_call.function.name
        function_args_str = tool_call.function.arguments
        function_args = json.loads(function_args_str)
        console.print(f"  - Tool: {function_name}, Args: {function_args_str}")
        if function_name == "add_numbers":
            tool_result_content = add_numbers(**function_args)

console.print(f"Final Result: {tool_result_content}")

LLM requested a tool call:

- Tool: add_numbers, Args: {"a": 77, "b": 11}

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f725-841e-7009-983a-ebe3579ab54f


Final Result: 88

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f725-e442-72b5-8fe3-5e232803a9f7


We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [13]:
messages.append(response.model_dump())

In [14]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": str(tool_result_content)
})

In [15]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant use tools to answer questions.'},
 {'role': 'user',
  'content': 'My lucky numbers are 77 and 11. What is their sum?'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'chatcmpl-tool-6f223df227274e9fb786d71a22744087',
    'function': {'arguments': '{"a": 77, "b": 11}', 'name': 'add_numbers'},
    'type': 'function'}],
  'reasoning_content': None},
 {'tool_call_id': 'chatcmpl-tool-6f223df227274e9fb786d71a22744087',
  'role': 'tool',
  'name': 'add_numbers',
  'content': '88'}]

You should have a sequence of messages like this:


In [16]:
[m["role"] for m in messages]

['system', 'user', 'assistant', 'tool']

Now call the model again with the new messages and it will use the tool call result to answer the question

In [17]:
final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.print(final_response.content)

The sum of your lucky numbers, 77 and 11, is 88.

## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [18]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a tool-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": inspect.getdoc(func).split("\\n")[0] if inspect.getdoc(func) else "",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }

    docstring = inspect.getdoc(func)
    param_descriptions = {}
    if docstring:
        args_section = False
        current_param = None
        for line in docstring.split('\\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped:
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" "): # Heuristic: end of args section
                     args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle List[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by List or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: OpenAI schema doesn't officially use default, but useful metadata

        schema["function"]["parameters"]["properties"][name] = param_schema
        if is_required:
            schema["function"]["parameters"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [19]:
tool_schema = generate_tool_schema(add_numbers)
console.print(tool_schema)

{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [20]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        console.print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [21]:
add_numbers = function_tool(add_numbers)
console.print(add_numbers.tool_schema)


{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

In [22]:
add_numbers.tool_schema

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': ''},
    'b': {'type': 'integer', 'description': ''}},
   'required': ['a', 'b']}}}

and call the tool =)

In [23]:
add_numbers(1, 2)

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f725-a3e0-7098-9df1-15a49e8ab476


3

### 3.1 Real Example using an API based tool

We are going to use the [EXA search API](https://docs.exa.ai/reference/getting-started).
- How does [EXA search works](https://docs.exa.ai/reference/how-exa-search-works#how-exa-search-works)
- Using exa search [as tool calling](https://docs.exa.ai/reference/openai-tool-calling)

In [24]:
query = "Recipes for cooking seabass?"

search_res = exa_client.search_and_contents(query=query, type='auto', num_results=4)

In [25]:
console.print(search_res)

SearchResponse(
    results=[
        Result(
            url='https://www.masterclass.com/articles/how-to-cook-sea-bass',
            id='https://www.masterclass.com/articles/how-to-cook-sea-bass',
            title='How to Cook Sea Bass: Simple Mediterranean Sea Bass Recipe - 2025 - MasterClass',
            score=None,
            published_date='2025-03-05T00:00:00.000Z',
            author='',
            image=None,
            favicon='https://www.masterclass.com/_next-public/favicon.ico',
            subpages=None,
            extras=None,
            text='Food Written by MasterClass Last updated: Mar 5, 2025 • 1 min read Mediterranean sea bass is an 
easy-to-cook fish that’s simple enough for a weeknight and luxurious enough for a special occasion. Learn From the 
Best Jump To Section What to Serve With Mediterranean Sea Bass Simple Mediterranean Sea Bass Recipe Gordon Ramsay 
Teaches Cooking I What to Serve With Mediterranean Sea Bass Like most white fish, sea bass has a mild flavor that 
pairs well with a variety of sides. Try this Mediterranean sea bass with a bowl of marinated kalamata olives, 
Mediterranean couscous salad, avocado salad, feta cheese marinated in olive oil and spices, or roasted 
Mediterranean vegetables such as bell peppers, cherry tomatoes, eggplant, and zucchini. Meet One of Your New 
Instructors Simple Mediterranean Sea Bass Recipe makes Ingredients 1 Heat a large skillet (preferably nonstick or 
well-seasoned cast iron) over medium-high heat. Add olive oil. Once shimmering, place sea bass filets skin side 
down into the pan. Cook until the skin is crispy and light brown, about 4 minutes. Flip to continue cooking until 
flesh is opaque and flaky, about 4 minutes longer. 2 Remove sea bass filets from the pan using a fish spatula and 
plate skin-side-up to keep skin crispy. 3 Make a quick pan sauce by deglazing the pan with dry white wine. Add 
capers, salt, black pepper, paprika, and minced garlic cloves. Keep sauce at a simmer, stirring occasionally, until
liquid has reduced by about half, about 2–4 minutes. Remove the sauce from the heat and quickly whisk in butter. 
Gently stir in fresh herbs. 4 Serve pan-seared sea bass filets with lemon wedges and sauce on the side, to be 
drizzled over the fish as desired. Become a better chef with the MasterClass Annual Membership. Gain access to 
exclusive video lessons taught by culinary masters, including Yotam Ottolenghi, Gabriela Cámara, Chef Thomas 
Keller, Massimo Bottura, Dominique Ansel, Gordon Ramsay, Alice Waters, and more. Featured MasterClass Instructor 
Gordon Ramsay Take your cooking to the next level in Gordon’s first MasterClass on essential methods, ingredients, 
and recipes. Explore the class',
            highlights=None,
            highlight_scores=None,
            summary=None
        ),
        Result(
            url='https://www.bbcgoodfood.com/recipes/collection/sea-bass-recipes',
            id='https://www.bbcgoodfood.com/recipes/collection/sea-bass-recipes',
            title='Sea bass recipes',
            score=None,
            published_date='2025-03-04T17:18:17.000Z',
            author='',
            image='https://images.immediate.co.uk/production/volatile/sites/30/2013/05/sea-bass-ea84f07.jpg?resize=
1200%2C630',
            favicon=None,
            subpages=None,
            extras=None,
            text="[![](https://images.immediate.co.uk/production/volatile/sites/30/2024/03/cropped-padded-GF-footer
-logo-988784d-e9bbf17.png?quality=90&resize=120,27)](https://www.bbcgoodfood.com)\n\n[![](https://images.immediate.
co.uk/production/volatile/sites/30/2024/03/cropped-GF-new-teal-1-7004649-a80b70d.png?quality=90&resize=265,65)](htt
ps://www.bbcgoodfood.com)\n\n![Pan-fried sea 
bass](https://images.immediate.co.uk/production/volatile/sites/30/2013/05/sea-bass-ea84f07.jpg?quality=90&resize=55
6,505)\n\n# Sea bass recipes\n\n36 Recipes\n\n[Get 5 issues for £5 when you subscribe to our 
magazine](https://www.buysubscri

Let's explore the payload

In [26]:
console.md("\n-------------------\n".join(result.text for result in search_res.results))

Food Written by MasterClass Last updated: Mar 5, 2025 • 1 min read Mediterranean sea bass is an easy-to-cook fish 
   that’s simple enough for a weeknight and luxurious enough for a special occasion. Learn From the Best Jump To   
   Section What to Serve With Mediterranean Sea Bass Simple Mediterranean Sea Bass Recipe Gordon Ramsay Teaches    
  Cooking I What to Serve With Mediterranean Sea Bass Like most white fish, sea bass has a mild flavor that pairs  
      well with a variety of sides. Try this Mediterranean sea bass with a bowl of marinated kalamata olives,      
      Mediterranean couscous salad, avocado salad, feta cheese marinated in olive oil and spices, or roasted       
   Mediterranean vegetables such as bell peppers, cherry tomatoes, eggplant, and zucchini. Meet One of Your New    
 Instructors Simple Mediterranean Sea Bass Recipe makes Ingredients 1 Heat a large skillet (preferably nonstick or 
  well-seasoned cast iron) over medium-high heat. Add olive oil. Once shimmering, place sea bass filets skin side  
 down into the pan. Cook until the skin is crispy and light brown, about 4 minutes. Flip to continue cooking until 
 flesh is opaque and flaky, about 4 minutes longer. 2 Remove sea bass filets from the pan using a fish spatula and 
  plate skin-side-up to keep skin crispy. 3 Make a quick pan sauce by deglazing the pan with dry white wine. Add   
capers, salt, black pepper, paprika, and minced garlic cloves. Keep sauce at a simmer, stirring occasionally, until
 liquid has reduced by about half, about 2–4 minutes. Remove the sauce from the heat and quickly whisk in butter.  
   Gently stir in fresh herbs. 4 Serve pan-seared sea bass filets with lemon wedges and sauce on the side, to be   
  drizzled over the fish as desired. Become a better chef with the MasterClass Annual Membership. Gain access to   
   exclusive video lessons taught by culinary masters, including Yotam Ottolenghi, Gabriela Cámara, Chef Thomas    
 Keller, Massimo Bottura, Dominique Ansel, Gordon Ramsay, Alice Waters, and more. Featured MasterClass Instructor  
Gordon Ramsay Take your cooking to the next level in Gordon’s first MasterClass on essential methods, ingredients, 
                                          and recipes. Explore the class                                           

🌆 ]8;id=720054;https://www.bbcgoodfood.com\cropped-padded-GF-footer-logo-988784d-e9bbf17.png?quality=90&resize=120,27]8;;\                                      

🌆 ]8;id=225594;https://www.bbcgoodfood.com\cropped-GF-new-teal-1-7004649-a80b70d.png?quality=90&resize=265,65]8;;\                                              

🌆 ]8;id=40184;https://images.immediate.co.uk/production/volatile/sites/30/2013/05/sea-bass-ea84f07.jpg?quality=90&resize=556,505\Pan-fried sea bass]8;;\                                                                                              

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                Sea bass recipes                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

36 Recipes                                                                                                         

]8;id=675490;https://www.buysubscriptions.com/print/good-food-magazine-subscription?promo=GFCAT24\Get 5 issues for £5 when you subscribe to our magazine]8;;\                                                             

Whether you serve it baked, grilled, pan-fried or in a fish pie, make sea bass the perfect dinner party dish or    
simple supper with these easy recipes                                                                              

For more recipes using delicious white fish, try our ]8;id=173850;https://www.bbcgoodfood.com/recipes/collection/white-fish-recipes\white fish recipes]8;;\, ]8;id=84695

In [27]:
@weave.op 
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.
    
    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.
    
    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.
    
    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)
    
    output = []
    for result in search_results.results:
        output.append(
            {"title": result.title,
            "text": result.text,
            "url": result.url
            }
        )
    return output

    

In [28]:
exa_search.tool_schema

{'type': 'function',
 'function': {'name': 'exa_search',
  'description': 'Perform a search query on the web and retrieve the most relevant URLs and web content.\n\nThis function uses the Exa search API to find relevant web pages based on the query\nand returns their titles, text content, and URLs.\n\nArgs:\n    query: The search query. Use detailed, specific queries for better results.\n           The quality of results depends on the specificity of the query.\n    num_results: The number of search results to retrieve. Defaults to 5.\n\nReturns:\n    A list of dictionaries, each containing:\n        - title: The title of the web page\n        - text: The text content of the web page\n        - url: The URL of the web page',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string', 'description': ''},
    'num_results': {'type': 'integer', 'description': '', 'default': 5}},
   'required': ['query']}}}

We get a list of results with the relevant metadata.

In [29]:
search_results = exa_search("How do I cook seabass?")
console.print(search_results)

[
    {
        'title': 'The BEST way to Cook Chilean Sea Bass that takes it to the NEXT ...',
        'text': '# [_Facebook_](https://www.facebook.com/)\n\n| | |\n| --- | --- |\n| Email or phone | Password 
|\n| [Forgot account?](https://www.facebook.com/recover/initiate?lwv=100&ars=royal_blue_bar) |\n\n[Create new 
account](https://www.facebook.com/r.php?locale=en_US)\n\n## You’re Temporarily Blocked\n\n## You’re Temporarily 
Blocked\n\nIt looks like you were misusing this feature by going too fast. You’ve been temporarily blocked from 
using it.\n\n[Back](https://www.facebook.com/www.facebook.com)\n\n- English (US)\n- 
[Español](https://www.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-
way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- [Français 
(France)](https://es-la.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-bes
t-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[中文(简体)](https://fr-fr.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-
best-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[العربية](https://zh-cn.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-bes
t-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- [Português 
(Brasil)](https://ar-ar.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-bes
t-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[Italiano](https://pt-br.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-be
st-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[한국어](https://it-it.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best
-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[Deutsch](https://ko-kr.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-bes
t-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[हिन्दी](https://de-de.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best-wa
y-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n- 
[日本語](https://hi-in.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Fchefbillyparisi%2Fposts%2Fthe-best
-way-to-cook-chilean-sea-bass-that-takes-it-to-the-next-level%2F1239156604237561%2F)\n\n- [Sign 
Up](https://www.facebook.com/reg/)\n- [Log In](https://www.facebook.com/login/)\n- 
[Messenger](https://messenger.com/)\n- [Facebook Lite](https://www.facebook.com/lite/)\n- 
[Video](https://www.facebook.com/watch/)\n- [Meta Pay](https://about.meta.com/technologies/meta-pay)\n- [Meta 
Store](https://www.meta.com/)\n- [Meta Quest](https://www.meta.com/quest/)\n- [Ray-Ban 
Meta](https://www.meta.com/smart-glasses/)\n- [Meta AI](https://www.meta.ai/)\n- [Meta AI more 
content](https://www.meta.ai/pages/what-is-labubu/?utm_source=foa_web_footer)\n- 
[Instagram](https://l.facebook.com/l.php?u=https%3A%2F%2Fwww.instagram.com%2F&h=AT0-jEAtr7zT126tLWKVRA4hvnWz4eo9c9M
a3SnO8b6Kcj5yAY6G-X4XWuEItQo8e9xxX2xa-WUiz7YohLnNWuIJHWsqUOCFd44idFwg3em1Is04uCqQg-4LBgQh5yokgLWyzzY6_nY8xhMdse1Z0H
pCQ1dl7q29O8RkjQ)\n- [Threads](https://www.threads.com/)\n- [Voting Information 
Center](https://www.facebook.com/votinginformationcenter/?entry_point=c2l0ZQ%3D%3D)\n- [Privacy 
Policy](https://www.facebook.com/privacy/policy/?entry_point=facebook_page_footer)\n- [Consumer Health 
Privacy](https://www.facebook.com/privacy/policies/health/?entry_point=facebook_page_footer)\n- [Privacy 
Center](https://www.facebook.com/privacy/center/?entry_point=facebook_page_footer)\n- 
[About](https://about.meta.com/)

weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f725-ad81-7d85-be36-50fef70abed3


In [30]:
messages = [
    {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
    {"role": "user", "content": "How do I cook seabass?"}]

response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[exa_search.tool_schema])
console.print(response)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-3a66a179986b42d39393ea8728623720',
            function=Function(
                arguments='{"query": "how to cook seabass step by step", "num_results": 5}',
                name='exa_search'
            ),
            type='function'
        )
    ],
    reasoning_content=None
)

Let's create some helper functions to perform the tool calls

In [31]:
from openai.types.chat.chat_completion_message_function_tool_call import ChatCompletionMessageFunctionToolCall

def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], tool_calls: list[ChatCompletionMessageFunctionToolCall]) -> list[dict]:
    "Perform the tool calls and return the messages with the tool call results"
    messages = []
    if not tool_calls:
        return messages
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        with console.status(f"[bold cyan]Executing {function_name}...[/bold cyan]"):
            tool = get_tool(tools, function_name)
            tool_response = tool(**function_args) # doesn't handle async
        
        # Create panel content
        panel_content = f"[bold cyan]🔧 Tool Call:[/bold cyan] {function_name}\n\n"
        panel_content += f"[dim]Args: {tool_call.function.arguments}[/dim]\n\n"
        
        if isinstance(tool_response, list):
            panel_content += f"[green]✓[/green] Found {len(tool_response)} results"
        else:
            panel_content += f"[green]✓[/green] {function_name} executed successfully"
        
        console.print(Panel(panel_content, border_style="cyan"))
        
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "content": str(tool_response),
        })
    return messages

In [32]:

# add the tool call result to the messages
messages.append(response.model_dump())
messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
messages.append({
    "role": "user",
    "content": "Answer my previous query based on the search results.",})

final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.rule("Final Model Response")
console.md(final_response.content)


weave: 🍩 https://wandb.ai/wandb-applied-ai-team/london-workshop-2025/r/call/0199f725-b03b-760f-993a-35ea3ce6ce7f


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "how to cook seabass step by step", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

Cooking seabass can be done in several delicious ways, depending on the cut and your preferred method. Here are    
some popular and easy techniques based on the search results:                                                      

                                     1. Pan-Seared Chilean Sea Bass (Stovetop)                                     

This method creates a golden, crispy crust and tender, flaky interior.                                             

Steps:                                                                                                             

 • Pat the fillets dry with a paper towel to ensure a good sear.                                                   
 • Season one side with salt and pepper.                                                                           
 • Melt butter in a hot non-stick or cast-iron skillet over medium-high heat. Add minced garlic and sauté for 1    
   minute.                                                                                                         
 • Place the fish seasoned-side down and cook for 2–3 minutes until golden. Flip and cook another 2–3 minutes until
   the fish is opaque and flakes easily.                                                                           
 • Remove from pan, drizzle with pan juices, and finish with a squeeze of fresh lemon.                             

Tip: For best results, sear skin-side down first if your fillets have skin. Use a meat thermometer—fish is done at 
145°F (63°C).                                                                                                      

▌ Source: ]8;id=280970;https://thebigmansworld.com/chilean-sea-bass-recipe/\The Big Man's World]8;;\                                                                                    

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                             2. Baked Sea Bass (Oven)                                              

A simple, hands-off method perfect for thick fillets.                                                              

Steps:                                                                                                             

 • Preheat oven to 400°F (200°C).                                                                                  
 • Let the fish sit at room temperature for 20 minutes for even cooking.                                           
 • Pat dry, brush with melted butter, and season with salt, pepper, and optional paprika.                          
 • Place on a greased baking dish and bake for 12–15 minutes (depending on thickness) until the internal           
   temperature reaches 145°F.                                                                                      
 • Finish with a squeeze of lemon juice.                                                                           

Tip: Thicker fillets may need a few extra minutes. Avoid overcooking to keep the fish moist.                       

▌ Source: ]8;id=841083;https://www.cookedbyjulie.com/baked-chilean-sea-bass/\Cooked by Julie]8;;\                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                 3. Pan-Fried Whole Sea Bass (Mediterranean Style)                                 

Ideal for a whole fish, offering crispy skin and flavorful flesh.                                                  

Steps:                                                                                                             

 • Clean and scale the fish if not already done. Pat dry.                                                          
 • Heat olive oil in a large frying pan until very hot.                                                            
 • Place 

Let's wrap this in a function:

In [33]:
@weave.op
def research(query: str) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
        {"role": "user", "content": query}]

    # call model with tools
    response = call_model(
        model_name=MODEL_SMALL, 
        messages=messages, 
        tools=[exa_search.tool_schema])

    # add the response to the messages
    messages.append(response.model_dump())

    # perform the tool calls
    messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
    
    # prompt the model to be grounded
    messages.append({"role": "user","content": "Answer my previous query based on the search results.",})

    final_response = call_model(model_name=MODEL_SMALL, messages=messages)
    return final_response.content

In [34]:
result = research("What are the most popular pokemons?")
console.md(result)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "most popular Pokémon of all time", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Based on the search results, the most popular Pokémon vary depending on the context—whether it's fan votes, trading
card game (TCG) prices, regional preferences, or general recognition. Here's a comprehensive summary:              

                                           Overall Most Popular Pokémon                                            

 • Charizard is frequently cited as the most popular Pokémon overall. A 2022 Reddit poll of over 55,000 users      
   ranked Charizard as the #1 favorite, receiving 1,107 votes in a large fan survey. It also dominates in online   
   communities like Serebii.net, with 3,105 users listing it as their favorite.                                    
 • Pikachu, while globally iconic and the franchise mascot, ranks lower in fan preference polls (e.g., 44th in one 
   Reddit survey) but remains the most recognizable Pokémon worldwide.                                             
 • Eevee is also extremely popular, especially among female fans, and ranks highly in multiple analyses.           

                         Top 10 Most Popular Pokémon (Based on Fan Surveys & Recognition)                          

  1 Charizard – Top favorite in multiple polls, beloved for its design, strength, and role in the anime.           
  2 Gengar – Ranked #2 in a major Reddit survey with 1,056 votes; its spooky yet cute design has gained massive    
    popularity.                                                                                                    
  3 Arcanine – Ranked #3 with 923 votes; admired for its majestic appearance and legendary status rumors.          
  4 Bulbasaur – Ranked #4 with 710 votes; the most popular starter Pokémon among fans.                             
  5 Blaziken – A favorite from Generation III, known for its aggressive battle style.                              
  6 Umbreon – Highly popular in the TCG and among fans of Dark-type Pokémon.                                       
  7 Lucario – Ranked #7 in fan votes and won second place in a 2020 official "Pokémon of the Year" poll.           
  8 Gardevoir – Especially popular among younger fans and known as a "waifu" favorite.                             
  9 Eevee – One of the most versatile and beloved Pokémon due to its multiple evolutions.                          
 10 Dragonite – A fan-favorite Dragon-type, especially loved in the U.S.                                           

                                                 Notable Mentions                                                  

 • Greninja – Won the official 2020 Pokémon of the Year fan vote with over 140,000 votes, thanks to its role in the
   anime as Ash’s partner.                                                                                         
 • Mimikyu – Placed #3 in the 2020 fan vote, driven by its emotional backstory and Pikachu-like disguise.          
 • Typhlosion, Blastoise, Snorlax, and Flygon also appear frequently in top rankings.                              

                                               Regional Preferences                                                

 • United States, UK, Australia: Prefer Charizard.                                                                 
 • Canada: Favors Pikachu over Charizard.                                                                          
 • UK: Shows strong love for Umbreon.                                                                              
 • Australia: Includes Mew and Arcanine in the top favorites.                                                      

                                        Trading Card Game (TCG) Popularity                                         

 • Charizard remains the most valuable and sought-after card, especially the 1999 first-edition holofoil, which    
   sold for $220,000.                                                                                              
 • Umbreon and Gengar have seen surges in TCG 

![](images/04_pokedex.png)

This is "Almost" an agent, but it's missing the loop. Let's add that next.

## 4. Implementing a Basic Agentic Loop

Let's implement a basic agentic loop. We'll use the `pokedex` function we just created. The implementation we have above has some limitations:
- Its a single turn, so if it fails to answer my question in one pass it is over.

![](images/05_agent.png)

From the really good [Anthropic Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article and encourage people to read it.

A simple for loop

In [35]:
@weave.op
def research_loop(query: str, max_turns: int = 4, tools = [exa_search, add_numbers]) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format."},
        {"role": "user", "content": query}]
    
    for turn in range(max_turns):
        console.rule(f"Agent Loop Turn {turn + 1}/{max_turns}")

        # call model with tools
        response = call_model(
            model_name=MODEL_MEDIUM, 
            messages=messages, 
            tools=[t.tool_schema for t in tools])

        # add the response to the messages
        messages.append(response.model_dump())

        # if the LLM requested tool calls, perform them
        if response.tool_calls:
            # perform the tool calls
            tool_outputs = perform_tool_calls(tools=[exa_search, add_numbers], tool_calls=response.tool_calls)
            messages.extend(tool_outputs)
        # LLM gave content response
        elif response.content:
            console.rule("Final Model Response")
            console.md(response.content)
            return response.content
        else:
            print("LLM response had neither content nor tool calls. Stopping loop.")
            break

In [36]:
res = research_loop("What is the sum of the populations of the 2 major EU cities?")

─────────────────────────────────────────────── Agent Loop Turn 1/4 ───────────────────────────────────────────────

InternalServerError: Error code: 500 - {'error': {'message': 'EngineCore encountered an issue. See stack trace (above) for the root cause.', 'type': 'Internal Server Error', 'param': None, 'code': 500}}

# 5. Structuring the Agent with Classes

The loop above works, but for more complex agents, encapsulating the logic and state within classes is much better. We'll define:
- `AgentState`: A Pydantic model to hold the conversation history and potentially other state.
- `SimpleAgent`: A class containing the agent's configuration (model, system message, tools) and logic (`step`, `run`).

In [ ]:
class AgentState(BaseModel):
    """Manages the state of the agent."""
    messages: List[Dict[str, Any]] = Field(default_factory=list)
    step: int = Field(default=0)
    final_assistant_content: str | None = None # Populated at the end of a run

In [ ]:
class SimpleAgent:
    """A simple agent class with tracing, state, and tool processing."""
    def __init__(self, model_name: str, system_message: str, tools: List[Callable]):
        self.model_name = model_name
        self.system_message = system_message
        self.tools = [function_tool(t) for t in tools] # add schemas to the tools
    
    @weave.op(name="SimpleAgent.step") # Trace each step
    def step(self, state: AgentState) -> AgentState:
        step = state.step + 1
        messages = state.messages
        final_assistant_content = None
        try:
            # call model with tools
            response = call_model(
                model_name=self.model_name, 
                messages=messages, 
                tools=[t.tool_schema for t in self.tools])

            # add the response to the messages
            messages.append(response.model_dump())

            # if the LLM requested tool calls, perform them
            if response.tool_calls:
                # perform the tool calls
                tool_outputs = perform_tool_calls(tools=self.tools, tool_calls=response.tool_calls)
                messages.extend(tool_outputs)

            # LLM gave content response
            else:
                messages.append(response.model_dump())
                final_assistant_content = response.content
        except Exception as e:
            console.print(f"ERROR in Agent Step: {e}")
            # Add an error message to history to indicate failure
            messages.append({"role": "assistant", "content": f"Agent error in step: {str(e)}"})
            final_assistant_content = f"Agent error in step {step}: {str(e)}"
        return AgentState(messages=messages, step=step, final_assistant_content=final_assistant_content)

    @weave.op(name="SimpleAgent.run")
    def run(self, user_prompt: str, max_turns: int = 10) -> AgentState:
        state = AgentState(messages=[
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": user_prompt}])
        for _ in range(max_turns):
            console.rule(f"Agent Loop Turn {state.step+1}/{max_turns}")
            state = self.step(state)
            if state.final_assistant_content:
                return state
        return state


![](images/07_simple_agent.png)

In [ ]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?")
print(f"Final response: {state.final_assistant_content}")


────────────────────────────────────────────── Agent Loop Turn 1/10 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "Ash Ketchum first three Pokémon names and weights", "num_results": 5}                          │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 2/10 ───────────────────────────────────────────────

Final response: Based on the information from the search results, Ash's first three Pokémon are:

1. **Pikachu** - Weight: 13.2 lbs (6.0 kg)
2. **Caterpie** - Weight: 6.4 lbs (2.9 kg)
3. **Bulbasaur** - Weight: 15.2 lbs (6.9 kg)

To find the combined weight, we add the weights together:

- 13.2 lbs (Pikachu) + 6.4 lbs (Caterpie) + 15.2 lbs (Bulbasaur) = **34.8 lbs**

In kilograms:
- 6.0 kg (Pikachu) + 2.9 kg (Caterpie) + 6.9 kg (Bulbasaur) = **15.8 kg**

Thus, the combined weight of Ash's first three Pokémon is **34.8 lbs (15.8 kg)**.


In [ ]:
@weave.op 
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search_and_refine(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.
    
    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.
    
    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.
    
    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)
    
    @weave.op
    def refine_search_result(result, query):
        messages = [
            {"role":"system", "content": f"Your task is to extract from the search results only the info that is relevant to answer the query"},
            {"role": "user", "content": f"- query: {query}\n- Search result: {result}"}
        ]
        refined_search = call_model(model_name=MODEL_SMALL, messages=messages)
        return refined_search.content

    output = []
    for item, result in enumerate(search_results.results):
        console.print(f"Refining result {item+1}")
        refined_text = refine_search_result(result.text, query)
        output.append(
            {"title": result.title,
            "text": refined_text,
            "url": result.url
            }
        )
    return output

In [ ]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search_and_refine, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?")
print(f"Final response: {state.final_assistant_content}")

────────────────────────────────────────────── Agent Loop Turn 1/10 ───────────────────────────────────────────────

Refining result 1

Refining result 2

Refining result 3

Refining result 4

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: add_numbers                                                                                       │
│                                                                                                                 │
│ Args: {"a": 15.0, "b": 18.5}                                                                                    │
│                                                                                                                 │
│ ✓ add_numbers executed successfully                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 4/10 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: add_numbers                                                                                       │
│                                                                                                                 │
│ Args: {"a": 33.5, "b": 19.6}                                                                                    │
│                                                                                                                 │
│ ✓ add_numbers executed successfully                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 5/10 ───────────────────────────────────────────────

Final response: The combined weight of Ash's first three Pokémon—Bulbasaur, Charmander, and Squirtle—is **53.1 lbs** (approximately 24.1 kg).  

### Breakdown:
- **Bulbasaur**: 15.0 lbs  
- **Charmander**: 18.5 lbs  
- **Squirtle**: 19.6 lbs  

**Total**: 15.0 + 18.5 + 19.6 = **53.1 lbs**

*Source: [Bulbapedia - List of Pokémon by weight](https://bulbapedia.bulbagarden.net/wiki/List_of_Pokémon_by_weight)*


Possible improvements to the SimpleAgent:
- Give the model info about the state of the conversation, you could inject a message with the model context pressure, steps left, etc.
- Structured output. Make the model output a specific format, for instance a JSON with the expected fields.
- Add more tools like read and write files, access a database.
- Agent handoff: Agent1 does triage and Agent2 executes specific tasks.

## Evaluating our SimpleAgent

In [ ]:
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from evaluation.eval import run_evaluation
from evaluation.eval_config import EvalConfig

In [ ]:
eval_config = EvalConfig(
    evaluation_name=f"SimpleAgent_{SimpleAgent.model_name}",
    trials=1,
    limit=1
    weave_parallelism=50,
    queries=project_root / "data/prompt_data/query.jsonl",
    reference=project_root / "data/test_data/cleaned_data/reference.jsonl",
    criteria=project_root / "data/criteria_data/criteria.jsonl",
)

results = await run_evaluation(
    eval_config=eval_config,
    agent_callable=agent.run,
)
results

50 dataset rows constructed for evaluation


────────────────────────────────────────────── Agent Loop Turn 1/10 ───────────────────────────────────────────────

Refining result 1

Refining result 2

Refining result 3

...

Refining result 3

Refining result 4

Refining result 5

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search_and_refine                                                                             │
│                                                                                                                 │
│ Args: {"query": "Duan Yongping Warren Buffett Charlie Munger investment philosophies", "num_results": 5}        │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 2/10 ───────────────────────────────────────────────